# Um roteiro utilizando algumas ferramentas do repositório ferramentas-adv-pje para extrair uma lista de processos do PJe do TJMT e extrair informações do processo por meio do Serviço MNI

## Primeiro, a construção da classe para acessar o PJe via Selenium.

In [2]:
import time
import csv
import json
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.select import Select
from selenium.webdriver.chromium.webdriver import ChromiumDriver

class Webdriver_PJe_TJMT_1(webdriver.Chrome):
    def __init__(self):
        super().__init__()
    frame_logar_ID = "ssoFrame"
    username_CSS_SELECTOR = "input#username"
    password_CSS_SELECTOR = "input#password"
    logar_ID = "kc-login"
    nome_parte_ID = "fPP:j_id162:nomeParte"
    outros_nomes_ID = "fPP:j_id171:outrosNomesAlcunha"
    nome_rep_ID = "fPP:j_id180:nomeAdvogado"
    cpf_ID = "fPP:dpDec:documentoParte"
    cnpj_ID = "cnpj"
    assunto_ID = "fPP:j_id248:assunto"
    classe_ID = "fPP:j_id257:classeJudicial"
    num_doc_ID = "fPP:j_id272:numeroDocumento"
    oab_ID = "fPP:decorationDados:numeroOAB"
    jurisd_ID_select = "fPP:jurisdicaoComboDecoration:jurisdicaoCombo"
    orgao_ID_select = "fPP:orgaoJulgadorComboDecoration:orgaoJulgadorCombo"
    data_ini_ID = "fPP:dataAutuacaoDecoration:dataAutuacaoInicioInputDate"
    data_final_ID = "fPP:dataAutuacaoDecoration:dataAutuacaoFimInputDate"
    valor_ini_ID = "fPP:valorDaCausaDecoration:valorCausaInicial"
    valor_fim_ID = "fPP:valorDaCausaDecoration:valorCausaFinal"
    pesquisar_CSS_SELECTOR ="input[id='fPP:searchProcessos']"
    
    
    
    def logar(self, username,password):
        self.get("https://pje.tjmt.jus.br/pje/login.seam")
        time.sleep(2)
        frame_logar = self.find_element(By.ID, self.frame_logar_ID)
        self.switch_to.frame(frame_logar)
        self.find_element(By.CSS_SELECTOR, self.username_CSS_SELECTOR).send_keys(username)
        self.find_element(By.CSS_SELECTOR, self.password_CSS_SELECTOR).send_keys(password)
        time.sleep(0.5)
        self.find_element(By.ID, self.logar_ID).click()
        time.sleep(2)


    def ir_consulta(self):
        self.get("https://pje.tjmt.jus.br/pje/Processo/ConsultaProcesso/listView.seam")
        time.sleep(5)

    def inserir_dados(self, nome_parte="", outros_nomes="",
                        nome_rep="", cpf="", cnpj="", assunto="", classe="", num_doc="", oab="", jurisd="", orgao="",
                        data_inicial="",data_final="", valor_ini="", valor_fim="", usuario="", senha=""):
        parametros_elementos = {
        "nome_parte": self.find_element(By.ID, self.nome_parte_ID),
        "outros_nomes": self.find_element(By.ID,self.outros_nomes_ID),
        "nome_rep": self.find_element(By.ID,self.nome_rep_ID),
        "cpf": self.find_element(By.ID,self.cpf_ID),
        "cnpj": self.find_element(By.ID,self.cnpj_ID),
        "assunto": self.find_element(By.ID,self.assunto_ID),
        "classe": self.find_element(By.ID,self.classe_ID),
        "num_doc": self.find_element(By.ID,self.num_doc_ID),
        "oab": self.find_element(By.ID,self.oab_ID),
        "data_inicial": self.find_element(By.ID,self.data_ini_ID),
        "data_final": self.find_element(By.ID,self.data_final_ID),
        "valor_ini": self.find_element(By.ID,self.valor_ini_ID),
        "valor_fim": self.find_element(By.ID,self.valor_fim_ID)
    }
        for parametro, elemento in parametros_elementos.items():
            if locals()[parametro]:
                elemento.send_keys(locals()[parametro])
                time.sleep(1) #para evitar erros
        if jurisd != "":
            self.jurisd_ID_select.select_by_visible_text(jurisd)
            time.sleep(0.5)
        else:
            pass
        if orgao != "":
            self.orgao_ID_select.select_by_visible_text(orgao)
            time.sleep(0.5)
        else:
            pass


    def pesquisar(self):
        self.find_element(By.CSS_SELECTOR,
                                            self.pesquisar_CSS_SELECTOR).click()
        time.sleep(10)

## Aqui, a função para usar a classe acima estabelecida para salvar determinados processos em um json.

In [3]:
def down_lista_procs(nome_parte="", outros_nomes="",
                        nome_rep="", cpf="", cnpj="", assunto="", classe="", num_doc="", oab="", jurisd="", orgao="",
                        data_inicial="",data_final="", valor_ini="", valor_fim="", usuario="", senha="",delay=3):
    lista_procs = []
    driver = Webdriver_PJe_TJMT_1()
    driver.logar(usuario,senha)
    driver.ir_consulta()
    driver.inserir_dados(nome_parte, outros_nomes,
                        nome_rep, cpf, cnpj, assunto, classe, num_doc, oab, jurisd, orgao,
                        data_inicial,data_final, valor_ini, valor_fim)
    driver.pesquisar()
    #a seção abaixo serve para definir a quantidade de página - o que é necessário para usar for ao invés de while
    #vai-se para a ultima pagina, e coleta-se o texto do elemento que marca a página atual
    #o try/except é para lidar com quando só há uma página, pois a rotina pressupõe
    #que haja uma página diferente da atual após pesquisar pelos processos
    cont_pags = 0
    botao_ultima_pag = driver.find_elements(By.CSS_SELECTOR, ".rich-datascr-button")[-1]
    botao_ultima_pag.click()
    time.sleep(5)
    try:
        pagina_atual = driver.find_element(By.CSS_SELECTOR, ".rich-datascr-act")
        ultima_pag = int(pagina_atual.text)
        botao_primeira_pag = driver.find_elements(By.CSS_SELECTOR, ".rich-datascr-button")[0]
        botao_primeira_pag.click()
        time.sleep(8)
    except:
        ultima_pag = 1
    #início do loop, visitará todas as páginas e salvará os dados de todos os processos.
    while cont_pags < ultima_pag:
        tabela_procs = driver.find_element(By.ID, "fPP:processosTable:tb")
        for row in tabela_procs.find_elements(By.TAG_NAME, "tr"):
            proc = row.find_elements(By.TAG_NAME, "td")
            proc_cell = proc[1]
            orgao_cell = proc[3]
            data_autuacao_cell = proc[4]
            classe_cell = proc[5]
            # as outras info (polo ativo, polo passivo e última moviment. são melhor capturadas pela API MNI do PJe)
            nmr_proc = proc_cell.text
            orgao_txt = orgao_cell.text
            classe_jud = classe_cell.text
            dia, mes, ano = data_autuacao_cell.text.split("/")
            values = [nmr_proc, orgao_txt, dia, mes, ano, classe_jud]
            print(values) #colocar aqui o que fazer com os dados
            lista_procs.append(values)
        cont_pags += 1
        botao_proxima_pag = driver.find_elements(By.CSS_SELECTOR, ".rich-datascr-button")[-2]
        botao_proxima_pag.click()
        time.sleep(delay)
    driver.quit()
    return lista_procs

## Aqui, baixando os processos e salvando em um json.
* No exemplo, estou pegando todos os processos em que apareço como advogado e tenham sido distribuídos em 2024.

In [ ]:
lista = down_lista_procs(nome_parte="", outros_nomes="",
                        nome_rep="SAULO NIEDERLE PEREIRA", cpf="", cnpj="", assunto="", classe="", num_doc="", oab="", jurisd="", orgao="",
                        data_inicial="01/01/2024",data_final="31/12/2024", valor_ini="", valor_fim="", usuario="", senha="")
resultados = {"processos": lista}
resultado_path = './resultados.json'
with open(resultado_path, "a") as output_file:
    json.dump(resultados, output_file, indent=2)

## Com os processos localizados, agora é utilizar o Serviço MNI para puxar informações mais detalhadas.
* Aqui, estou me atendo a localizar informações de quem está no processo, quando foi distribuído, qual o valor da causa e onde está.

In [4]:
import csv
import requests
from bs4 import BeautifulSoup
import time
import random

class MNI_TJMT():
    def __init__(self, usuario, senha):
        #self.dict_assuntos = self.get_dict_assuntos()
        #self.dict_classes = self.get_dict_classes()
        self.usuario = usuario
        self.senha = senha
    x = y = 'false'
    URL = 'https://pje.tjmt.jus.br/pje/intercomunicacao' #conferir se funciona para segundo grau
    
   
    def get_dict_assuntos(self):
        DICT_CNJ_ASSUNTOS = {}
        with open(r"codigos_cnj/Códigos Assuntos CNJ.csv", mode='r') as f: #csv com os códigos de assuntos e seus significados
            csv_reader = csv.reader(f, delimiter=',')
            for row in csv_reader:
                row_split = row[0].split(";")
                DICT_CNJ_ASSUNTOS[row_split[0]] = row_split[1]
        return DICT_CNJ_ASSUNTOS
    
    def get_dict_classes(self):
        DICT_CNJ_CLASSES = {}
        with open(r"codigos_cnj/Códigos Classes CNJ.csv", mode='r') as f: #csv com os códigos de classes e seus significados
            csv_reader = csv.reader(f, delimiter=',')
            for row in csv_reader:
                row_split = row[0].split(";")
                DICT_CNJ_CLASSES[row_split[0]] = row_split[1]
        return DICT_CNJ_CLASSES

    def consulta_MNI(self, lista_procs, movimentos = False,documentos = False, delay=None):
        lista_return = []
        usuario = self.usuario
        senha = self.senha
        x = y = 'false'
        if movimentos:
            x = 'true'
        if documentos: 
            y = 'true'
        SOAPEnvelope_CONSULTAR_PROCESSO = f"""
                                                <soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/" xmlns:ser="http://www.cnj.jus.br/servico-intercomunicacao-2.2.2/" xmlns:tip="http://www.cnj.jus.br/tipos-servico-intercomunicacao-2.2.2">
                                                <soapenv:Header/>
                                                <soapenv:Body>
                                                <ser:consultarProcesso>
                                                <tip:idConsultante>[USUARIO]</tip:idConsultante>
                                                <tip:senhaConsultante>[SENHA]</tip:senhaConsultante>
                                                <tip:numeroProcesso>[PROCESSO]</tip:numeroProcesso>
                                                <tip:movimentos>{x}</tip:movimentos>
                                                <tip:incluirDocumentos>{y}</tip:incluirDocumentos>
                                                </ser:consultarProcesso>
                                                </soapenv:Body>
                                                </soapenv:Envelope>
                                                """
        self.SOAPEnvelope_CONSULTAR_PROCESSO= SOAPEnvelope_CONSULTAR_PROCESSO.replace("[USUARIO]",usuario).replace("[SENHA]",senha)
        
        for proc in lista_procs:
            tentativas = 0
            sucesso = False
            
            while tentativas < 10 and not sucesso:
                try:
                    #valores de interesse
                    #há muitos outros, por ora extraindo apenas os abaixo
                    valor_causa = None
                    codigo_localidade = None
                    classe = None
                    # não extraio data pois já vem de outro roteiro
                    # data_autuacao = None
                    lista_assuntos = []
                    orgao_julgador = None
                    lista_partes_at = []
                    lista_adv_at = []
                    lista_partes_pa = []
                    lista_adv_pa = []
                    x = self.SOAPEnvelope_CONSULTAR_PROCESSO.replace("[PROCESSO]",proc)
                    response = requests.post(self.URL, data=x)
                    #print(response)
                    xml_content = response.text
                    start_marker = '<soap:Envelope'
                    end_marker = '</soap:Envelope>'
                    start_pos = xml_content.find(start_marker)
                    end_pos = xml_content.find(end_marker) + len(end_marker)
                    body_content = xml_content[start_pos:end_pos]
                    #print(body_content)
                    soup = BeautifulSoup(body_content, 'xml')
                    dados_basicos = soup.find("dadosBasicos")
                    #print(dados_basicos)
                    dataAjuizamento = dados_basicos['dataAjuizamento']
                    #os elementos abaixo nem sempre estão presentes
                    #try:
                        #classe = self.dict_classes[dados_basicos['classeProcessual']]
                    #except:
                        #pass
                    try:
                        codigo_localidade = dados_basicos['codigoLocalidade']
                    except:
                        pass
                    try:
                        valor_causa = dados_basicos.find('valorCausa').getText()
                    except:
                        pass
                    try:
                        orgao_julgador = dados_basicos.find('orgaoJulgador')['nomeOrgao']
                    except:
                        pass
                    try:
                        assuntos = dados_basicos.find_all('assunto')
                        #for assunto in assuntos:
                            #lista_assuntos.append(self.dict_assuntos[assunto.getText()])
                    except:
                        pass
                    polos = dados_basicos.find_all('polo')
                    for polo in polos:
                        if polo['polo'] == 'AT':
                            partes_at = polo.find_all('ns2:parte')
                            for parte in partes_at:
                                pessoa = parte.find('ns2:pessoa')
                                # extraindo apenas o nome, sem interesse nos outros atributos
                                try:
                                    pessoa_atributos = {
                                    'nome': pessoa['nome'],
                                    'cpf': pessoa['numeroDocumentoPrincipal'],
                                    'sexo': pessoa['sexo']
                                    }
                                except:
                                    pessoa_atributos = {
                                    'nome': pessoa['nome'],
                                    'cpf': pessoa['numeroDocumentoPrincipal'],
                                    'sexo': None
                                    }
                                endereco = pessoa.find_all('ns2:endereco')[0]
                                try:
                                    cep = endereco['cep']
                                except:
                                    cep = None
                                try:
                                    logradouro = endereco.find_all('ns2:logradouro')[0].text
                                except:
                                    logradouro = None
                                try:
                                    numero = endereco.find_all('ns2:numero')[0].text
                                except:
                                    numero = None
                                try:    
                                    complemento = endereco.find_all('ns2:complemento')[0].text
                                except:
                                    complemento = None
                                try:
                                    bairro = endereco.find_all('ns2:bairro')[0].text
                                except:
                                    bairro = None
                                try:
                                    cidade = endereco.find_all('ns2:cidade')[0].text
                                except:
                                    cidade = None
                                try:
                                    estado = endereco.find_all('ns2:estado')[0].text
                                except:
                                    estado = None
                                info_endereco = [cep, logradouro, numero, complemento, bairro, cidade, estado]
            
                                lista_partes_at.append([pessoa_atributos['nome'],pessoa_atributos['cpf'],
                                                        pessoa_atributos['sexo'], info_endereco])
                                advogados = parte.find_all('ns2:advogado')
                                for advogado in advogados:
                                    #extraindo apenas o nome
                                    advogado_atributos = {
                                        'nome': advogado['nome']
                                        }
                                    lista_adv_at.append(advogado_atributos['nome'])
                        elif polo['polo'] == 'PA':
                            partes_pa = polo.find_all('ns2:parte')
                            for parte in partes_pa:
                                pessoa = parte.find('ns2:pessoa')
                                # extraindo apenas o nome
                                pessoa_atributos = {
                                    'nome': pessoa['nome']
                                }
                                lista_partes_pa.append(pessoa_atributos['nome'])
                                advogados = parte.find_all('ns2:advogado')
                                for advogado in advogados:
                                    advogado_atributos = {
                                        'nome': advogado['nome']
                                    }
                                    lista_adv_pa.append(advogado_atributos['nome'])
                    valores = [proc,  lista_partes_at, lista_partes_pa, dataAjuizamento, orgao_julgador, valor_causa]
                    print(valores) #colocar aqui o que fazer com os valores
                    lista_return.append(valores)
                    sucesso = True
                    
                except Exception as e:
                    tentativas += 1
                    print(f"Erro no processo {proc} - tentativa {tentativas}/10: {e}")
                    if tentativas < 10:
                        time.sleep(5)
                    else:
                        print(f"Processo {proc} falhou após 10 tentativas")
                        
            #atraso randomizado
            if delay:
                r = random.uniform(0.2,1)
                time.sleep(r + delay)
                
        return lista_return

mni = MNI_TJMT("CPF_advogado","senha")

In [ ]:
with open(r'C:\Users\saulo\GitHub\ferramentas-adv-pje\resultados.json', 'r') as file:
    data = json.load(file)
    
LISTA_PROCS = [x[0] for x in data['processos']]
consulta = mni.consulta_MNI(LISTA_PROCS, movimentos = False,documentos = False, delay=0.5)

## Expande uma lista onde o índice [1] contém sublistas,criando uma nova lista para cada sublista no índice [1].
 ### Tive que fazer isso pois, da maneira coma consulta MNI está sendo feita, se houver mais de um pessoa no polo ativo, o resultado é uma lista com sublistas no índice[1]. Com isto, eu expando a lista, e depois posso reunir todos do mesmo processo pelo número CNJ.

In [ ]:
def expandir_lista(lista_original):
    # Extrai os elementos
    primeiro_elemento = lista_original[0]
    sublistas = lista_original[1]
    elementos_restantes = lista_original[2:]
    
    # Cria uma nova lista para cada sublista no índice [1]
    listas_expandidas = []
    
    for sublista in sublistas:
        nova_lista = [primeiro_elemento, [sublista]] + elementos_restantes
        listas_expandidas.append(nova_lista)
    
    return listas_expandidas

consulta_expandida = []
for x in range(0, len(consulta)):
    x_expandido = expandir_lista(consulta[x])
    consulta_expandida.extend(x_expandido)

## Preparando para o DataFrame

In [ ]:
lista_dicts = []
for item in consulta_expandida:
    dic = {
        'numeroprocesso': item[0],
        'nome': item[1][0][0],
        'cpf': item[1][0][1],
        'sexo': item[1][0][2],
        'cep': item[1][0][3][0],
    'logradouro': item[1][0][3][1],
    'numero': item[1][0][3][2],
    'complemento': item[1][0][3][3],
    'bairro': item[1][0][3][4],
    'cidade': item[1][0][3][5],
    'estado': item[1][0][3][6],
    'passivo': item[2],
    'dataajuizamento': item[3][:8],
    'orgao': item[4],
    'valorcausa': item[5]
    }
    lista_dicts.append(dic)

In [ ]:
import pandas as pd

df = pd.DataFrame(lista_dicts)
df.info()